# Paper Exp: Compress 2D
- This notebook contains:
  -  compression training (from scratch) using `train.py`;
  -  compressed model generation using `codec.py`;
  -  metric calculation:
     -  brightfield image ssim/psnr/corr;
     -  fluorescent prediction image ssim/psnr/corr;
     -  compression ratio; 

In [15]:
# import modules:
from pathlib import Path
import pandas as pd
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
import numpy as np
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter
from pathlib import Path
import os
import sys
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import monai
import torch
import random
from datetime import datetime
import shutil

In [16]:
# define funcs and hyper-parameters:

_SEED = 2023
np.random.seed(_SEED)
random.seed(_SEED)
os.environ["PYTHONHASHSEED"] = str(_SEED)


class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def normalizeItensity(image):
    # Convert the image data to a floating-point data type
    img = image.astype(np.float32)
    # Convert the image to a toch Tensor
    img = torch.from_numpy(img)
    # Normalize the intensity of the image using the MONAI NormalizeIntensity transform
    normalize_intensity = monai.transforms.NormalizeIntensity()
    img_normalized = normalize_intensity(img)
    # Convert the normalized image back to a numpy array
    return img_normalized.numpy()


def transform_img(image):
    img = image.astype(np.float32) / 65535
    return img


def compare_images(path1, path2, gt=True):
    # Load the two images
    image1 = AICSImage(path1).get_image_data("YX")
    image2 = AICSImage(path2).get_image_data("YX")
    if gt:
        # image1 = image1.astype(np.float32)
        # # Scale the values in image1 to the range [0, 65535]
        # scaled_image = (image1 / np.max(image1) * 65535).round().astype(np.uint16)
        # image1 = scaled_image
        image1 = transform_img(image1)
        image2 = transform_img(image2)
    # Calculate metrics
    mse = np.sum((image1 - image2) ** 2) / (924 * 624)
    ssim_value = ssim(image1, image2, data_range=1)
    psnr_value = psnr(image1, image2, data_range=1)
    corr = np.corrcoef(image1.ravel(), image2.ravel())[0, 1]
    # psnr = 10 * np.log10(1 / (mse + 0.000001))
    return mse, ssim_value, psnr_value, corr

In [17]:
# firstly, use fine_tune_v12 to generate compressed images, and calculate metrics:
test_data_path = sorted(
    Path(
        "/mnt/data/ISAS.DE/yu.zhou/Yu/project/data_compression/data/labelfree_2d/v2/test"
    ).glob("*.tiff")
)
random.seed(_SEED)
test_data_path = random.sample(test_data_path, 50)
compress_data_path = Path.cwd()
metrics = ["ms-ssim"]
models = ["bmshj2018-factorized"]

In [ ]:
copy_path = Path(
    "/mnt/eternus/users/Yu/project/data_compression/experiment/paper_exp/2d/compression/raw_img"
)
for tmp_path in test_data_path:
    shutil.copy(tmp_path, copy_path)

In [ ]:
for model in models:
    for metric in metrics:
        for image in test_data_path: 
            model_name= model+'_'+metric+'_'+'8_v12'
            path_encoded= compress_data_path/model_name/str(image.stem+'_encoded')
            path_encoded.parent.mkdir(exist_ok=True, parents = True)
            path_decoded= compress_data_path/model_name/str(image.stem+'_decoded.tiff')
            path_decoded.parent.mkdir(exist_ok=True, parents = True)
            if not path_decoded.is_file():
                !python3 ../../../codec.py encode {image} -o {path_encoded} --model {model} -q 8 -m {metric} --cuda --checkpoint {path_encoded.parent / "model.pth.tar"}
                # !python3 ../codec.py encode {image} -o {path_encoded} --model bmshj2018-factorized -q 8 -m {metric} --cuda
                !python3 ../../../codec.py decode {path_encoded} -o {path_decoded} --model {model} -q 8 -m {metric} --cuda --checkpoint {path_encoded.parent / "model.pth.tar"}

# Evaluation:

1. brightfield image metric: mse/ssim/psnr/corr:

In [21]:
for model in models:
    for metric in metrics:
        model_name = model + "_" + metric + "_" + "8_v12"
        decoded_paths = sorted((compress_data_path / model_name).glob("*.tiff"))
        mse_value = AverageMeter()
        ssim_value = AverageMeter()
        psnr_value = AverageMeter()
        corr_value = AverageMeter()
        for i, (decode_path, raw_path) in enumerate(
            zip(decoded_paths, sorted(test_data_path))
        ):
            tmp_mse, tmp_ssim, tmp_psnr, tmp_corr = compare_images(
                decode_path, raw_path
            )
            mse_value.update(tmp_mse)
            ssim_value.update(tmp_ssim)
            psnr_value.update(tmp_psnr)
            corr_value.update(tmp_corr)
        print(
            "MSE:",
            mse_value.avg,
            "SSIM:",
            ssim_value.avg,
            "PSNR:",
            psnr_value.avg,
            "CORR:",
            corr_value.avg,
        )
        # Define the filename
        filename = Path.cwd() / model_name / "note.txt"
        # Open the file for writing (creating if it doesn't exist)
        with open(filename, "a") as file:
            # Write the values to the file
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            file.write(f"[Timestamp]: {timestamp}\n")
            file.write(f"MSE: {mse_value.avg}\n")
            file.write(f"SSIM: {ssim_value.avg}\n")
            file.write(f"PSNR: {psnr_value.avg}\n")
            file.write(f"Correlation: {corr_value.avg}\n")

MSE: 0.00022894785484657378 SSIM: 0.9769605329180757 PSNR: 38.54215274176078 CORR: 0.9203892596547221
